In [1]:
import pandas as pd
import numpy as np

In [2]:
movie_data = pd.read_csv("G:\\Case study\\Recommendation Engine\\KNN Movie Recommendation\\movies.csv")
rating_data = pd.read_csv("G:\\Case study\\Recommendation Engine\\KNN Movie Recommendation\\ratings.csv")
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movie_data.drop(['genres'],axis=1, inplace = True)
movie_data.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
rating_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
dataset = rating_data.merge(movie_data,on = 'movieId')
dataset.head()

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story (1995)
1,5,1,4.0,847434962,Toy Story (1995)
2,7,1,4.5,1106635946,Toy Story (1995)
3,15,1,2.5,1510577970,Toy Story (1995)
4,17,1,4.5,1305696483,Toy Story (1995)


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 5 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
title        100836 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 4.6+ MB


In [7]:
movie_rating = dataset.dropna(axis=0,subset=['title'])
movie_rating_count = (movie_rating.
                     groupby(by=['title'])['rating'].
                     count().
                     reset_index().
                     rename(columns={'rating':'totalRatingCount'})
                     [['title','totalRatingCount']])
movie_rating_count.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [8]:
rating_with_totalCount =dataset.merge(movie_rating_count, left_on='title',right_on='title',how='left')
rating_with_totalCount.head()

,userId,movieId,rating,timestamp,title,totalRatingCount
0,1,1,4.0,964982703,Toy Story (1995),215
1,5,1,4.0,847434962,Toy Story (1995),215
2,7,1,4.5,1106635946,Toy Story (1995),215
3,15,1,2.5,1510577970,Toy Story (1995),215
4,17,1,4.5,1305696483,Toy Story (1995),215


In [9]:
rating_threshold = 50
rating_popular_movie = rating_with_totalCount.query('totalRatingCount >= @rating_threshold')
rating_popular_movie.head()

,userId,movieId,rating,timestamp,title,totalRatingCount
0,1,1,4.0,964982703,Toy Story (1995),215
1,5,1,4.0,847434962,Toy Story (1995),215
2,7,1,4.5,1106635946,Toy Story (1995),215
3,15,1,2.5,1510577970,Toy Story (1995),215
4,17,1,4.5,1305696483,Toy Story (1995),215


In [10]:
rating_popular_movie.shape

(41362, 6)

In [11]:
#Making the Pivot table
pivot_userID = rating_popular_movie.pivot_table(index='title',columns='userId', values ='rating').fillna(0)
pivot_userID.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [12]:
from scipy.sparse import csr_matrix
movie_features_matrix  = csr_matrix(pivot_userID.values)
#Creating model called KNearestNeighbors
from sklearn.neighbors import NearestNeighbors

knn_model =NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(movie_features_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [13]:
movie_features_matrix.shape

(450, 606)

In [14]:
select_movie = np.random.choice(pivot_userID.shape[0])
print(select_movie)
distance, indices = knn_model.kneighbors(pivot_userID.iloc[select_movie,:].values.reshape(1, -1), n_neighbors= 6)

406


In [15]:
for i in range(0, len(distance.flatten())):
    if i == 0:
        print('Recommendation for {0}: \n'.format(pivot_userID.index[select_movie]))
    else:
        print('{0}:{1}, with distance of {2}:'.format(i, pivot_userID.index[indices.flatten()[i]], distance.flatten()[i]))

Recommendation for Top Gun (1986): 

1:Hunt for Red October, The (1990), with distance of 0.46792656028553203:
2:Indiana Jones and the Temple of Doom (1984), with distance of 0.48809053862887464:
3:Star Wars: Episode VI - Return of the Jedi (1983), with distance of 0.5059958176148713:
4:Die Hard (1988), with distance of 0.5100477282419693:
5:Men in Black (a.k.a. MIB) (1997), with distance of 0.512490560175586:
